In [118]:
# Imports

import os
import requests
import json
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
import mistune

from langchain import OpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.tools import BaseTool
from langchain.llms.base import LLM
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage
from langchain_core.outputs import ChatGeneration

In [94]:
# Setup

load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

if not OPENROUTER_API_KEY:
    raise ValueError("Please set the OPENROUTER_API_KEY environment variable.")

API_URL = "https://openrouter.ai/api/v1/chat/completions"

HEADERS = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
}

In [95]:
# Single Agent

def call_openrouter(message, model):

    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": message}
        ]
    }
    
    response = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload))

    if response.status_code == 200:
        data = response.json()
        if "choices" not in data:   
            raise Exception(f"Unexpected response format: {data}")
        return data["choices"][0]["message"]["content"]

        return data["choices"][0]["message"]["content"]
    else:
        raise Exception(f"Request failed with status {response.status_code}: {response.text}")



def agent(prompt, model):

    print("Agent starting...")
    print("Sending prompt:", prompt)
    
    try:
        response_text = call_openrouter(prompt, model)
        display(Markdown(response_text))

        markdown = mistune.create_markdown(renderer='ast')
        tokens = markdown(response_text)
        # print(tokens)
                
    except Exception as e:
        print("\nError during API call:\n", e)


In [96]:
# 2 Agents

def conversation_session(agent1_model, agent2_model, initial_message, num_turns=5):
   
    conversation_history = []
    current_message = initial_message
    
    print("Starting conversation...\n")
    print("Agent1:", current_message)

    conversation_history.append(("Agent1", current_message))
    

    for turn in range(num_turns):
        prompt_for_agent2 = "The conversation so far:\n"
        for speaker, msg in conversation_history:
            prompt_for_agent2 += f"{speaker}: {msg}\n"

        prompt_for_agent2 += "\nAgent1 just spoke. Agent2, please respond."
        try:
            agent2_response = call_openrouter(prompt_for_agent2, agent2_model)
        except Exception as e:
            print(f"Error from Agent2: {e}")
            agent2_response = "[Error in Agent2 response]"

        print("Agent2:", agent2_response)
        conversation_history.append(("Agent2", agent2_response))
        
        prompt_for_agent1 = "The conversation so far:\n"
        for speaker, msg in conversation_history:
            prompt_for_agent1 += f"{speaker}: {msg}\n"

        prompt_for_agent1 += "\nAgent2 just spoke. Agent1, please respond."
        try:
            agent1_response = call_openrouter(prompt_for_agent1, agent1_model)
        except Exception as e:
            print(f"Error from Agent1: {e}")
            agent1_response = "[Error in Agent1 response]"

        print("Agent1:", agent1_response)
        conversation_history.append(("Agent1", agent1_response))
    
    return conversation_history


In [101]:
# Run 1-2 Agents 

gemini_2_flash_lite = 'google/gemini-2.0-flash-lite-preview-02-05:free'
gemini_2_pro_exp = 'google/gemini-2.0-pro-exp-02-05:free'
gemini_2_flash_thinking_exp = 'google/gemini-2.0-flash-thinking-exp:free'

prompt = 'Discuss building a short term trading strategy for BTCUSDT. Scalping is the goal on the 1-1hr time frames. The strategy should be based on the price and volume data not all these other indicators.\n'

# response = agent(prompt, gemini_2_flash_lite)
# history = conversation_session(gemini_2_pro_exp, gemini_2_pro_exp, prompt, num_turns=50)

In [119]:
# Langchain MultiAgent 

class OpenRouterLLM(LLM):
    openrouter_key: str
    model_name: str
    temperature: float = 0.0  

    @property
    def _llm_type(self) -> str:
        return "openrouter"

    def _call(self, prompt: str, stop=None, **kwargs) -> str:
        API_URL = "https://openrouter.ai/api/v1/chat/completions"
        HEADERS = {
            "Authorization": f"Bearer {self.openrouter_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature
        }

        response = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload))
        data = response.json()
        output_text = data["choices"][0]["message"]["content"]
        # Wrap into a ChatGeneration if needed for logging or debugging,
        # but return only the text:
        chat_output = ChatGeneration(message=AIMessage(content=output_text, role="assistant"))
        return chat_output.message.content

    @property
    def _identifying_params(self):
        return {
            "openrouter_key": self.openrouter_key,
            "model_name": self.model_name,
            "temperature": self.temperature
        }


llm = OpenRouterLLM(
    openrouter_key=OPENROUTER_API_KEY,
    model_name="google/gemini-2.0-flash-thinking-exp:free",
    temperature=0
)


In [116]:
# Agent Tools

# Tool to send emails (placeholder)
class EmailTool(BaseTool):
    name: str = "send_email"
    description: str = "Sends an email with a subject and body to a given recipient."

    def _run(self, subject: str, body: str, recipient: str) -> str:
        # Here you would integrate with your email API.
        return f"Email sent to {recipient} with subject '{subject}' and body '{body[:50]}...'."

    async def _arun(self, subject: str, body: str, recipient: str) -> str:
        raise NotImplementedError("Async not implemented")


# Tool to scrape web content
class WebScraperTool(BaseTool):
    name: str = "scrape_web"
    description: str = "Scrapes and returns plain text from the given URL."

    def _run(self, url: str) -> str:
        from bs4 import BeautifulSoup
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text(separator="\n")  # Return text with line breaks

    async def _arun(self, url: str) -> str:
        raise NotImplementedError("Async not implemented")


# Tool to retrieve YouTube video details (placeholder)
class YouTubeTool(BaseTool):
    name: str = "scrape_youtube"
    description: str = "Retrieves information about a YouTube video given its URL."

    def _run(self, url: str) -> str:
        # In production, use the YouTube Data API or a dedicated library.
        return f"Details for YouTube video at {url} (dummy data)."

    async def _arun(self, url: str) -> str:
        raise NotImplementedError("Async not implemented")


# Tool to generate a report from data
class ReportGeneratorTool(BaseTool):
    name: str = "generate_report"
    description: str = "Generates a summary report from the provided data string."

    def _run(self, data: str) -> str:
        # In production, you might use templating or ask the LLM to generate a report.
        return f"Report Summary: {data[:100]}..."  # Dummy summary using first 100 characters

    async def _arun(self, data: str) -> str:
        raise NotImplementedError("Async not implemented")



tools = [EmailTool(), WebScraperTool(), YouTubeTool(), ReportGeneratorTool()]

In [ ]:
# Run MultiAgent 

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a multi-tool agent that can perform a variety of tasks."),
    MessagesPlaceholder("chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad")
])

agent = create_openai_tools_agent(llm, tools, prompt=prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools)


query = (
    "Send an email with subject 'Daily Report' to [email protected] "
    "that summarizes the latest news from https://cnn.com."
)

result = agent_executor.invoke({"input": query})
print(result)